In [21]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import numpy as np
import pandas as pd
import itertools as it
import networkx as nx
import matplotlib.pyplot as plt


from finite_groups import *

from activation_funcs import *

In [22]:
import csv

rows = []
for a in range(3):
    for b in range(3):
        for c in range(3):
            for d in range(3):
                if (a*d - b*c) % 3 == 1:
                    rows.append([a,b,c,d])


with open("SL2F3.csv","w",newline="") as f:
    w = csv.writer(f)
    w.writerow(["a","b","c","d"])
    w.writerows(rows)


In [23]:
group_df = pd.read_csv("groups/SL2F5.csv")
subgroup_df = pd.read_csv("groups/SL2F3.csv")


In [ ]:
# ==============================================================================
# SECTION 1: DEFINE GROUP
# ==============================================================================
def define_group():
    """
    Define the elements of your group G and the multiplication rule.
    
    Returns:
        elements (list): A list of all group elements (hashable: tuples, strings, ints).
        mult_func (callable): A function f(a, b) -> a * b.
    """

    elements = group_df[['a', 'b', 'c', 'd']].to_numpy().reshape(-1, 2, 2)

    def mult_func(p, q):
        return (p @ q) % 5

    return elements, mult_func

# ==============================================================================
# SECTION 2: DEFINE SUBGROUP
# ==============================================================================
def define_subgroup(subgroup_elements):
    """
    Define the subgroup H. 
    You must return a list of elements that are present in the 'group_elements' list.
    """

    H = subgroup_df[['a', 'b', 'c', 'd']].to_numpy().reshape(-1, 2, 2)
    
    return H

# ==============================================================================
# SECTION 3: DEFINE CONJUGACY CLASSES (MANUAL)
# ==============================================================================
def define_conjugacy_classes():
    """
    Manually define the conjugacy classes of G.
    Returns a list of tuples: (Representative, [List of all Members])
    
    If you return None, the solver will auto-compute them (computationally expensive).
    """
    

    # ------------------------------------------------------------
    # 1. Identity
    # ------------------------------------------------------------
    c1_rep = (1, 0, 0, 1)
    c1_members = [(1, 0, 0, 1)]

    # ------------------------------------------------------------
    # 2. Central element -I
    # ------------------------------------------------------------
    c2_rep = (4, 0, 0, 4)   # -I mod 5
    c2_members = [(4, 0, 0, 4)]

    # ------------------------------------------------------------
    # 3–4. Order 3 elements (two distinct classes)
    # ------------------------------------------------------------
    c3_rep = (0, 4, 1, 4)   # trace = -1
    c4_rep = (4, 4, 1, 0)   # trace = -1, not conjugate

    # Members are large (20 each) – normally generated algorithmically
    c3_members = "20 elements (order 3)"
    c4_members = "20 elements (order 3)"

    # ------------------------------------------------------------
    # 5. Order 4 elements
    # ------------------------------------------------------------
    c5_rep = (0, 4, 1, 0)
    c5_members = "30 elements (order 4)"

    # ------------------------------------------------------------
    # 6–7. Order 5 elements (two inverse classes)
    # ------------------------------------------------------------
    c6_rep = (1, 1, 1, 2)
    c7_rep = (2, 4, 4, 1)

    c6_members = "12 elements (order 5)"
    c7_members = "12 elements (order 5)"

    # ------------------------------------------------------------
    # 8. Order 6 elements
    # ------------------------------------------------------------
    c8_rep = (0, 1, 4, 1)
    c8_members = "20 elements (order 6)"

    # ------------------------------------------------------------
    # 9. Order 10 elements
    # ------------------------------------------------------------
    c9_rep = (2, 1, 1, 3)
    c9_members = "4 elements (order 10)"

    return [
        (c1_rep, c1_members),
        (c2_rep, c2_members),
        (c3_rep, c3_members),
        (c4_rep, c4_members),
        (c5_rep, c5_members),
        (c6_rep, c6_members),
        (c7_rep, c7_members),
        (c8_rep, c8_members),
        (c9_rep, c9_members),
    ]

# ==============================================================================
# SECTION 4: CHARACTER TABLE
# ==============================================================================
def define_character_table():
    """
    Define the Character Table.
    Returns:
        char_map (dict): { Representative : [chi_1, chi_2, ...] }
        labels (list): Names of the irreps
    
    NOTE: The keys in char_map MUST match the 'Representative' elements
          you defined in SECTION 3.
    """

    import cmath
    
        # ---- Cyclotomic values used by GAP ----
    # In GAP: A = -E(5) - E(5)^4, and *A is its complex conjugate. :contentReference[oaicite:2]{index=2}
    z = cmath.exp(2j * cmath.pi / 5)   # E(5)
    A = -(z + z**4)
    Aconj = A.conjugate()              # = -(z**2 + z**3)

    # ---- Class representatives (explicit matrices in SL(2,5)) ----
    rep_1a  = (1, 0, 0, 1)             # order 1
    rep_10a = (0, 1, 4, 3)             # order 10
    rep_10b = (3, 4, 1, 0)             # inverse of rep_10a, order 10
    rep_2a  = (4, 0, 0, 4)             # -I, order 2 (central)
    rep_5a  = (4, 3, 2, 3)             # (rep_10a)^2, order 5
    rep_5b  = (3, 2, 3, 4)             # inverse of rep_5a, order 5
    rep_3a  = (0, 1, 4, 4)             # order 3
    rep_6a  = (0, 1, 4, 1)             # order 6
    rep_4a  = (0, 1, 4, 0)             # order 4

    # ---- Character values by class (columns) from GAP display ----
    # Display(CharacterTable(SL(2,5))) lines X.1..X.9. :contentReference[oaicite:3]{index=3}
    # We convert "." to 0.
    # Each value list is [X.1, X.2, X.3, X.4, X.5, X.6, X.7, X.8, X.9] at that class.
    char_map = {
        rep_1a:  [1,  2,  2,  3,  3,  4,  4,  5,  6],
        rep_10a: [1,  A,  Aconj, Aconj, A,  -1,  1,  0, -1],
        rep_10b: [1,  Aconj, A,  A,  Aconj, -1,  1,  0, -1],
        rep_2a:  [1, -2, -2,  3,  3,  4, -4,  5, -6],
        rep_5a:  [1, -A, -Aconj, Aconj, A,  -1, -1,  0,  1],
        rep_5b:  [1, -Aconj, -A,  A,  Aconj, -1, -1,  0,  1],
        rep_3a:  [1, -1, -1,  0,  0,  1,  1, -1,  0],
        rep_6a:  [1,  1,  1,  0,  0,  1, -1, -1,  0],
        rep_4a:  [1,  0,  0, -1, -1,  0,  0,  1,  0],
    }

    labels = [
        "X.1 (deg 1)",
        "X.2 (deg 2)",
        "X.3 (deg 2)",
        "X.4 (deg 3)",
        "X.5 (deg 3)",
        "X.6 (deg 4)",
        "X.7 (deg 4)",
        "X.8 (deg 5)",
        "X.9 (deg 6)",
    ]

    return char_map, labels



# ==============================================================================
# MAIN EXECUTION
# ==============================================================================
def run_analysis():
    print(">>> 1. Loading Group Data...")
    elements, mult_func = define_group()
    classes_data = define_conjugacy_classes()
    
    # Initialize Group with MANUAL classes
    G = FiniteGroup(elements, mult_func, classes=None)
    print(f"    Group Order: {G.n}")
    
    print("\n>>> 2. Loading Subgroup Data...")
    H_elements = define_subgroup(elements)
    solver = InducedRepSolver(G)
    solver.set_subgroup(H_elements)
    
    print("\n>>> 3. Loading Character Table...")
    char_map, labels = define_character_table()
    solver.load_character_table(char_map, irrep_labels=labels)
    
    print("\n>>> 4. Computing Exact Projectors (SymPy)...")
    solver.compute_projectors()
    
    print("\n>>> 5. Induced Representation Decomposition:")
    for label, Q in solver.Qblocks.items():
        if Q.shape[1] > 0:
            print(f"    Irrep '{label}' appears with dimension {Q.shape[1]}")
    
    print("\n>>> 6. Building Interaction Graph...")
    
    # Define activation function using SymPy (e.g. Max(0, x) for ReLU)
    def relu_sym(x): 
        return sp.Max(0, x)
    
    graph = solver.build_interaction_graph(activation_fn=relu_sym, verbose=True)
    print(f"    Edges found: {graph.edges()}")
    
    if len(graph.nodes) > 0:
        plt.figure(figsize=(6, 6))
        pos = nx.spring_layout(graph, seed=42)
        nx.draw_networkx(graph, pos, node_color="#E8F0FF", edgecolors="blue", 
                         node_size=1000, font_weight="bold", with_labels=True)
        plt.title(f"Interaction Graph (SymPy Exact)")
        plt.show()

In [25]:
run_analysis()

>>> 1. Loading Group Data...


TypeError: define_conjugacy_classes() missing 1 required positional argument: 'subgroup_elements'